# Import Required Libraries
Import the necessary libraries, including pandas and Rasa.

In [23]:
import pandas as pd
import yaml

# Load Dataset
Load the dataset containing sentences using pandas.

In [4]:
# Load the dataset containing sentences using pandas
df_sentences = pd.read_csv('./data/scrapped/sentences.csv')

# Display the first few rows of the dataframe to verify the data
df_sentences.head()

,sentenceID,sentence,label,departure,destination
0,1,Réserver un billet pour Nantes,TRIP_ORDER,NaN,Nantes
1,2,Aller à Lyon pour une réunion,TRIP_ORDER,NaN,Lyon
2,3,This is not French,NOT_FRENCH,NaN,NaN
3,4,Réserver un hôtel à Bordeaux,NOT_TRIP,NaN,NaN
4,5,Visiter la Tour Eiffel,TRIP_ORDER,NaN,Paris


# Preprocess Data
Preprocess the data to prepare it for training the Rasa model.

In [26]:
def extract_departure_and_destination(row):
    text = row['sentence']
    departure = row['departure']
    destination = row['destination']
    if isinstance(departure, str) and isinstance(text, str):
        text = text.replace(departure, '[' + departure + '](destination)')
    if isinstance(destination, str) and isinstance(text, str):
        text = text.replace(destination, '[' + destination + '](destination)')
    row['sentence'] = text
    return row

# Appliquer les modifications sur chaque ligne
data = df_sentences.apply(extract_departure_and_destination, axis=1)

In [32]:
# Créer une structure pour le fichier YAML
nlu_data = {"version": "3.0", "nlu": []}

# Ajouter les phrases à la structure YAML
for label in data['label'].unique():
    examples = data[data['label'] == label]['sentence'].dropna().tolist()
    print(examples)
    intent_entry = {
        "intent": label.lower(),
        "examples": "|\n  - " + "\n  - ".join(examples)
    }
    nlu_data["nlu"].append(intent_entry)

# Exporter le fichier YAML
output_file = "nlu.yml"
with open(output_file, "w", encoding="utf-8") as file:
    yaml.dump(nlu_data, file, allow_unicode=True, sort_keys=False)

print(f"Le fichier {output_file} a été généré avec succès !")


['Réserver un billet pour [Nantes](destination)', 'Aller à [Lyon](destination) pour une réunion', 'Visiter la Tour Eiffel', 'Prendre le train pour [Marseille](destination)', 'Je veux aller de [Brest](destination) à [Lyon](destination)', 'je suis à [Marseille](destination) et je veux aller à [Rennes](destination)', 'Je dois me rendre à [Nice](destination) demain', 'Rendez-vous à [Marseille](destination) pour un concert', 'Organiser un voyage à [Toulouse](destination)', "J'ai réservé un vol pour [Montpellier](destination)", 'Aller à [Strasbourg](destination) pour les fêtes', 'Je pars de [Tours](destination) pour aller à [Nice](destination)', 'Démarrer de [Lyon](destination) pour rejoindre [Marseille](destination)', 'Prendre le train de [Lille](destination) à [Toulouse](destination)', 'Je dois prendre un vol de [Nantes](destination) à [Nice](destination)', 'Je vais de [Marseille](destination) à [Lyon](destination) pour un séminaire', 'Je pars de [Lille](destination) à [Strasbourg](destina